In [2]:
#basic
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from tqdm import tqdm, tqdm_notebook
#for text-preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack, coo_matrix
#hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
#stacking
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
#models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import lightgbm as lgbm

In [ ]:
train = pd.read_csv('toxic_train.csv')
test = pd.read_csv('toxic_train.csv')
labels = pd.read_csv('labels_tr')

In [ ]:
labels.head()

## Comments cleaning:

In [ ]:
def beautify_text(s):
    s = s.replace(':)', 'smile')
    s = s.replace(':D', 'laugh')
    # Removes all characters from string except letters and digits and convert letters to lowercase
    s = re.sub("[^a-zA-Z0-9]", " ", s.lower())
    # Remove new lines
    s = re.sub("\\n","", s)
    # Change urls
    s = re.sub("[a-zA-Z0-9]*(https://)[a-zA-Z0-9.]*", "url", s)
    # Remove IPs
    s = re.sub("\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}","", s)
    # Remove usernames
    s = re.sub("\[\[.*\]","", s)
    # Remove numbers
    s = re.sub("\\b[0-9]+\\b", "", s)
    # better use (\w+[0-9])
    # fix grammar
    s = re.sub('\\bf{1,}[a-z]*(ck)\\b', 'fuck', s)
    # find cock symb lol
    s = re.sub('8={1,}D?', 'cock', s)

    #formalize text
    s = s.replace("don\'t", "do not")
    s = s.replace("doesn\'t", "does not")
    s = s.replace("didn\'t", "did not")
    s = s.replace("won\'t", "will not")
    s = s.replace("haven\'t", "have not")
    s = s.replace('haven\\\'t', 'have not')
    #s = s.replace('\'ll', ' will')
    #clean text
    s = s.replace('\\', ' ')
    s = s.replace('\n', '')
    s = s.replace("'s", ' is')
    s = s.replace('“', '')
    s = s.replace('fck', 'fuck')
    s = s.replace('wtf', 'what the fuck')
    s = s.replace('cockwad', 'idiot')
    s = s.replace('скоморохъ', 'buffoon')
    s = s.replace('backash', 'backlash')
    s = s.replace('bizarrethere', 'bizarre there')
    s = s.replace('backash', 'backlash')
    s = s.replace('dummasses', 'dumbass')
    s = s.replace('lesbianswomen', 'lesbian women')
    s = s.replace('bisexualis', 'bisexual')
    s = s.replace('sexualitypenis','sexuality penis')
    s = s.replace('dorkgasm', 'orgasm')
    s = s.replace('cockmy', 'cock my')
    s = s.replace('supertroll', 'super troll')
    s = s.replace('faggotttttttt', 'faggot')
    s = s.replace('sucks50', 'sucks 50')
    s = s.replace('themfuck', 'them fuck')
    s = s.replace('offenderfugitive', 'offender fugitive')
    s = s.replace('creationisrael', 'creation israel')
    s = s.replace('assholestating', 'asshole stating')
    s = s.replace('idiotenough', 'idiot enough')
    s = s.replace('lessblunt', 'less blunt')
    s = s.replace('anticanadian', 'anti canadian')
    s = s.replace('afrikanblack', 'african black')
    s = s.replace('againbitch', 'again bitch')
    s = s.replace('animalfucker', 'animal fucker')
    return s

In [ ]:
train.comment_text = train.comment_text.apply(beautify_text)
test.comment_text = test.comment_text.apply(beautify_text)

In [ ]:
neg_words = [i[:-1] for i in open('negative-words.txt')]

symbs = ['!', '@', '#', '$', '%', '^', '&', '(', '*', '№', '=', '-', '+', '<', '>']
punct = ['.', ',', '\'', ';', ':']
smailes = [':)', ':D', ')']
neg_phrases = ['fuck you', 'screw you', 'kill you', 'beat you', 'die you', 'damn it', 'damn you', 'bitchin out', 'duck face', 'heil hitler' ]
short_phrases = ['wtf', 'dgaf', 'fml', 'foad', 'ftw', 'milf']
races_and_orients = ['lesbian', 'gay', 'homo', 'bromance', 'dyke', 'lesbo', 'lezzie', 'faggot', 'nigga', 'nigger', 'jew', 'yankee', 'racist']

## Train features

In [ ]:
neg_words_counts = train_data.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in neg_words]))

symbols_count = train.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in symbs]))

punct_count = train.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in punct]))

smailes_count = train.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in smailes]))

len_of_comment = train.comment_text.apply(lambda a: len(a))

capital_letters = train.comment_text.apply(lambda a: len([i for i in a if i.isupper()]))

neg_phrases_count = train.comment_text.apply(lambda a: len([i for i in neg_phrases if i in a]))

short_phrases_count = train.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in short_phrases]))

races_and_orients_count = train.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in races_and_orients]))

count_spaces = train.comment_text.apply(lambda a: len([i for i in a if i == ' ']))

count_ips = train.comment_text.apply(lambda a: len(re.findall("\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}", a)))

train = train.assign(capital_letters=capital_letters,
                     len_of_comment=len_of_comment,
                     count_ips=count_ips,
                     count_spaces=count_spaces,
                     sybs = symbols_count,
                     smailes = smailes_count,
                     neg_words=neg_words_counts,
                    neg_phrases=neg_phrases_count,
                    short_phrases=short_phrases_count,
                    races_and_orients=races_and_orients_count)

## Test features

In [ ]:
neg_words_counts_ts = test.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in neg_words]))

capital_letters_ts = test.comment_text.apply(lambda a: len([i for i in a if i.isupper()]))
len_of_comment_ts = test.comment_text.apply(lambda a: len(a))
count_spaces_ts = test.comment_text.apply(lambda a: len([i for i in a if i == ' ']))
count_ips_ts = test.comment_text.apply(lambda a: len(re.findall("\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}", a)))
punct_count_ts = test.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in punct]))
symbols_count_ts = test.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in symbs]))
smailes_count_ts = test.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in smailes]))
neg_phrases_count_ts = test.comment_text.apply(lambda a: len([i for i in neg_phrases if i in a]))
short_phrases_count_ts = test.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in short_phrases]))
races_and_orients_count_ts = test.comment_text.apply(lambda a: len([i for i in a.lower().split() if i in races_and_orients]))

test = test.assign(capital_letters=capital_letters_ts,
                     len_of_comment=len_of_comment_ts,
                     count_ips=count_ips_ts,
                     count_spaces=count_spaces_ts,
                     sybs = symbols_count_ts,
                     smailes = smailes_count_ts,
                     neg_words=neg_words_counts_ts,
                    neg_phrases=neg_phrases_count_ts,
                    short_phrases=short_phrases_count_ts,
                    races_and_orients=races_and_orients_count_ts)

In [3]:
train = pd.read_csv('toxic_train.csv')
test = pd.read_csv('toxic_test.csv')
labels = pd.read_csv('labels_tr')

train.head()

,id,comment_text,capital_letters,count_ips,count_spaces,len_of_comment,neg_phrases,neg_words,races_and_orients,short_phrases,smailes,sybs
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,49,252,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m s...,0,0,29,102,0,1,0,0,0,0
2,000113f07ec002fd,hey man i m really not trying to edit war it...,0,0,47,233,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i can t make any real suggestions on im...,0,0,136,622,0,1,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember...,0,0,17,67,0,0,0,0,0,0


In [4]:
labels.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [5]:
extra_features = ['capital_letters', 'len_of_comment', 'count_ips', 'count_spaces', 'sybs', 'smailes', 'neg_words', 'neg_phrases', 'short_phrases', 'races_and_orients']
classes_ = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_ids = train[['id']]

In [6]:
tfv = TfidfVectorizer(min_df=50,  max_features=30000, 
            analyzer='char',ngram_range=(1,3))
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

tfv.fit(all_text)
train_text = tfv.transform(train.comment_text)
test_text = tfv.transform(test.comment_text)

tfv = TfidfVectorizer(min_df=50,  max_features=30000, 
            analyzer='word',ngram_range=(1,3))
tfv.fit(all_text)
train_text = hstack((train_text, tfv.transform(train.comment_text)))
test_text = hstack((test_text, tfv.transform(test.comment_text)))

In [7]:
test_text.shape

(153164, 39567)

In [8]:
# train_text_ = train_text.todense()
# test_text_ = test_text.todense()
for i in extra_features:
    mms = MinMaxScaler()
    train_text = hstack((train_text, coo_matrix(mms.fit_transform(train[i].values.reshape(-1, 1)))))
    test_text = hstack((test_text, coo_matrix(mms.transform(test[i].values.reshape(-1, 1)))))

/home/novikov/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
train_text.shape

(159571, 39577)

In [10]:
labels['toxic'].shape

(159571,)

In [11]:
train_text = train_text.tocsr()
test_text = test_text.tocsr()

In [12]:
import warnings
warnings.filterwarnings("ignore")

## SVM

In [21]:
def objective_svc(params):
    model = SVC(kernel=params['kernel'], max_iter=10)
    
    shuffle = KFold()
    score = cross_val_score(model, X_train, y_train, cv=shuffle, scoring='roc_auc', n_jobs=1)
    return 1-score.mean()

X_train = train_text

trials = Trials()

In [24]:
param_space = {
                'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
              }

In [29]:
best = []

for i in range(len(classes_)):
    y_train = labels.values[:, i]
    print('Fitting model...')
    best = fmin(objective_svc,
                param_space,
                algo=tpe.suggest,
                max_evals=5)
    print(classes_[i], 'best:')
    print(best)

Fitting model...
linear
rbf
linear
sigmoid
rbf
toxic best:
{'kernel': 3}
Fitting model...
poly
sigmoid
rbf
rbf
linear
severe_toxic best:
{'kernel': 3}
Fitting model...
poly
rbf
linear
linear
sigmoid
obscene best:
{'kernel': 0}
Fitting model...
linear
rbf
sigmoid
linear
sigmoid
threat best:
{'kernel': 0}
Fitting model...
poly
poly
linear
linear
poly
insult best:
{'kernel': 1}
Fitting model...
poly
sigmoid
rbf
rbf
linear
identity_hate best:
{'kernel': 2}


```best_svm = space_eval(param_space, best)
print('Best paramns for svm:', best_svm)```

## LightGBM Gradient Boosting

In [12]:
def objective_lgbm(params):
    params = {
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    model = lgbm.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.01,
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        **params
    )
    print(model)
    shuffle = KFold()
    score = cross_val_score(model, X_train, y_train, cv=shuffle, scoring='roc_auc', n_jobs=1)
    return 1-score.mean()

X_train = train_text

trials = Trials()

In [13]:
model = lgbm.LGBMClassifier()
param_space = {
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
}

for i in range(len(classes_)):
    y_train = labels.values[:, i]
    print('Fitting model...')
    best = fmin(objective_lgbm,
                param_space,
                algo=tpe.suggest,
                max_evals=4,
                trials=trials)
    print(classes_[i], 'best:')
    print(best)

Fitting model...
LGBMClassifier(boosting_type='gbdt', colsample_bytree='0.842',
        learning_rate=0.01, max_bin=255, max_depth=-1,
        min_child_samples=10, min_child_weight=5, min_split_gain=0.0,
        n_estimators=500, n_jobs=-1, num_leaves=54, objective=None,
        predictor='gpu_predictor', random_state=0, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1, tree_method='gpu_hist')
LGBMClassifier(boosting_type='gbdt', colsample_bytree='0.672',
        learning_rate=0.01, max_bin=255, max_depth=-1,
        min_child_samples=10, min_child_weight=5, min_split_gain=0.0,
        n_estimators=500, n_jobs=-1, num_leaves=36, objective=None,
        predictor='gpu_predictor', random_state=0, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1, tree_method='gpu_hist')
LGBMClassifier(boosting_type='gbdt', colsample_bytree='0.564',
        learning_rate=

In [16]:
best_lgbm = {'colsample_bytree': 0.564246261338949, 'num_leaves': 62}

## OOF algorithm

In [13]:
X_train = train_text
X_test = test_text

In [14]:
def stacking_1(model, folds=10, classes_to_fit=classes_):
    scores = []
    scores_classes = np.zeros((len(classes_to_fit), folds))

    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    
    if (len(classes_to_fit)>1):
        submission = pd.DataFrame.from_dict({'id': test['id']})
        submission_oof =  pd.DataFrame.from_dict({'id': train['id']})
    else:
        submission = pd.DataFrame()
        submission_oof =  pd.DataFrame()
    for j, (class_name) in enumerate(classes_to_fit):
        print('Picking a class...')
        
        classifier = model

        test_pred = np.zeros(X_test.shape[0])
        train_pred = np.zeros(labels[class_name].shape[0])

        
        for i, (train_index, val_index) in enumerate(skf.split(X_train, labels[class_name].values)):
           
            fold_train, fold_val = X_train[train_index], X_train[val_index]
            y_train, y_val = labels.loc[train_index], labels.loc[val_index]
            print('Fitting model...')
            classifier.fit(fold_train, y_train[class_name])
            
            train_pred[val_index] = classifier.predict_proba(fold_val)[:, 1]
            test_pred += classifier.predict_proba(X_test)[:, 1]
           
            scores_classes[j][i] = roc_auc_score(y_val[class_name], train_pred[val_index])
            
            print('\n Fold %02d class %s AUC: %.6f' % ((i+1), class_name, scores_classes[j][i]))
            
        train_oof_auc = roc_auc_score(labels[class_name], train_pred)
        print('\n Average class %s AUC:\t%.6f' % (class_name, np.mean(scores_classes[j])))
        print(' Out-of-fold class %s AUC (train):\t%.6f' % (class_name, train_oof_auc))
        
        submission[class_name] = test_pred / folds
        submission_oof['prediction_' + class_name] = train_pred
        
    return submission_oof, submission 

In [29]:
print('SVM')
kernels = ['sigmoid', 'sigmoid', 'linear', 'linear', 'poly', 'rbf']
test_svm = pd.DataFrame.from_dict({'id': test['id']})
train_svm = pd.DataFrame.from_dict({'id': train['id']})

train_svm[classes_[0]], test_svm[classes_[0]] = stacking_1(SVC(kernel=kernels[0], max_iter=10, probability=True), classes_to_fit=[classes_[0]])
for i in range(1, len(classes_)):
    train_svm[classes_[i]], test_svm[classes_[i]] = stacking_1(SVC(kernel=kernels[i], max_iter=10, probability=True), classes_to_fit=[classes_[i]])   

SVM
Picking a class...
Fitting model...

 Fold 01 class toxic AUC: 0.440339
Fitting model...

 Fold 02 class toxic AUC: 0.671019
Fitting model...

 Fold 03 class toxic AUC: 0.504968
Fitting model...

 Fold 04 class toxic AUC: 0.500440
Fitting model...

 Fold 05 class toxic AUC: 0.616368
Fitting model...

 Fold 06 class toxic AUC: 0.759078
Fitting model...

 Fold 07 class toxic AUC: 0.479310
Fitting model...

 Fold 08 class toxic AUC: 0.482598
Fitting model...

 Fold 09 class toxic AUC: 0.645885
Fitting model...

 Fold 10 class toxic AUC: 0.543730

 Average class toxic AUC:	0.564374
 Out-of-fold class toxic AUC (train):	0.542519
Picking a class...
Fitting model...

 Fold 01 class severe_toxic AUC: 0.335931
Fitting model...

 Fold 02 class severe_toxic AUC: 0.554124
Fitting model...

 Fold 03 class severe_toxic AUC: 0.719803
Fitting model...

 Fold 04 class severe_toxic AUC: 0.770820
Fitting model...

 Fold 05 class severe_toxic AUC: 0.285404
Fitting model...

 Fold 06 class severe_toxic

In [32]:
train_svm.to_csv('train_svm.csv')
test_svm.to_csv('test_svm.csv')

In [ ]:
print('GradientBoosting')
train_lgbm, test_lgbm = stacking_1(lgbm.LGBMClassifier(n_estimators=500,
        learning_rate=0.01,
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        **best_lgbm))

GradientBoosting
Picking a class...
Fitting model...

 Fold 01 class toxic AUC: 0.961265
Fitting model...

 Fold 02 class toxic AUC: 0.966141
Fitting model...

 Fold 03 class toxic AUC: 0.965902
Fitting model...

 Fold 04 class toxic AUC: 0.965012
Fitting model...

 Fold 05 class toxic AUC: 0.965764
Fitting model...

 Fold 06 class toxic AUC: 0.964430
Fitting model...


In [19]:
print('LogReg')
train_lr, test_lr = stacking_1(LogisticRegression())
print('MultinomialNB')
train_mnb, test_mnb = stacking_1(MultinomialNB())

LogReg
Picking a class...
Fitting model...

 Fold 01 class toxic AUC: 0.974020
Fitting model...

 Fold 02 class toxic AUC: 0.978608
Fitting model...

 Fold 03 class toxic AUC: 0.974214
Fitting model...

 Fold 04 class toxic AUC: 0.974892
Fitting model...

 Fold 05 class toxic AUC: 0.976137
Fitting model...

 Fold 06 class toxic AUC: 0.977330
Fitting model...

 Fold 07 class toxic AUC: 0.973775
Fitting model...

 Fold 08 class toxic AUC: 0.977807
Fitting model...

 Fold 09 class toxic AUC: 0.977316
Fitting model...

 Fold 10 class toxic AUC: 0.978905

 Average class toxic AUC:	0.976300
 Out-of-fold class toxic AUC (train):	0.976278
Picking a class...
Fitting model...

 Fold 01 class severe_toxic AUC: 0.986412
Fitting model...

 Fold 02 class severe_toxic AUC: 0.987077
Fitting model...

 Fold 03 class severe_toxic AUC: 0.989266
Fitting model...

 Fold 04 class severe_toxic AUC: 0.988387
Fitting model...

 Fold 05 class severe_toxic AUC: 0.991080
Fitting model...

 Fold 06 class severe_to

In [24]:
train_lr.to_csv('train_lr.csv')
test_lr.to_csv('test_lr.csv')

train_mnb.to_csv('train_mnb.csv')
test_mnb.to_csv('test_mnb.csv')